In [79]:
from C4I import benchmarks, clustering, evaluation, investment, risk_factors2, utils
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from numpy import corrcoef
from pandas import DataFrame
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout

In [4]:
path = 'Dataset'
X = pd.read_csv(f'{path}/Stock_Market.csv', index_col=0)
X.index = pd.to_datetime(X.index)
X.shape

(1955, 151)

In [19]:
scaler_pre = "MinMax"
X.index = pd.to_datetime(X.index)

# divide train test dataset

train_ratio = 0.8
split_index = int(len(X) * train_ratio)

X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]

# Monthly returns
XM = utils.DailyPrice2MonthlyReturn(X_train)
# Daily returns
XD = utils.DailyPrice2DailyReturn(X_train)
# Scale data
if scaler_pre == 'Standard':
    XD = pd.DataFrame( StandardScaler().fit_transform(XD.values),
                      index = XD.index, columns = XD.columns )
    XM = pd.DataFrame( StandardScaler().fit_transform(XM.values),
                      index = XM.index, columns = XM.columns )
elif scaler_pre == 'MinMax':
    XD = pd.DataFrame( MinMaxScaler().fit_transform(XD.values),
                      index = XD.index, columns = XD.columns )
    XM = pd.DataFrame( MinMaxScaler().fit_transform(XM.values),
                      index = XM.index, columns = XM.columns )

In [66]:
XM2 = utils.DailyPrice2MonthlyReturn(X_test)
# Daily returns
XD2 = utils.DailyPrice2DailyReturn(X_test)
XD2 = pd.DataFrame( MinMaxScaler().fit_transform(XD2.values),
                  index = XD2.index, columns = XD2.columns )
XM2 = pd.DataFrame( MinMaxScaler().fit_transform(XM2.values),
                  index = XM2.index, columns = XM2.columns )
dfs2 = [XD2, XM2]
gran_names=['Daily','Monthly']
threshold=0.5
SEED=1
n_pcs = 7

In [71]:
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [89]:
XD2.shape

(390, 151)

In [110]:
# nnpca on the daily data is easy to overfit, divide it into 2 models, on monthly is still the previous one
dfs = [XD, XM] 
dfs2 = [XD2,XM2]
gran_names=['Daily','Monthly']
threshold=0.5
SEED=1
n_pcss = [5,7]

Encoded = list()

i = 0
df = dfs[i]
name = gran_names[i]
n_pcs = n_pcss[i]
input_dim = df.shape[1]

# Define a deeper autoencoder architecture
input_layer = keras.layers.Input(shape=(input_dim,))
encoded_1 = keras.layers.Dense(16, activation='tanh', activity_regularizer=regularizers.l1(10**(-4)))(input_layer)
encoded_1 = Dropout(0.5)(encoded_1)
encoded_2 = keras.layers.Dense(8, activation='relu', activity_regularizer=regularizers.l1(10**(-4)))(encoded_1)
encoded_2 = Dropout(0.5)(encoded_2)
encoded_final = keras.layers.Dense(n_pcs, activation='relu', activity_regularizer=regularizers.l1(10**(-4)))(encoded_2)

# Symmetric decoder architecture
decoded_1 = keras.layers.Dense(8, activation='relu', activity_regularizer=regularizers.l1(10**(-4)))(encoded_final)
decoded_1 = Dropout(0.5)(decoded_1)
decoded_2 = keras.layers.Dense(16, activation='tanh', activity_regularizer=regularizers.l1(10**(-4)))(decoded_1)
decoded_2 = Dropout(0.5)(decoded_2)
decoded_final = keras.layers.Dense(input_dim, activation='linear', activity_regularizer=regularizers.l1(10**(-4)))(decoded_2)

autoencoder = keras.Model(input_layer, decoded_final)
encoder = keras.Model(input_layer, encoded_final)

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

autoencoder.compile(optimizer='SGD', loss='mean_squared_error')
autoencoder.fit(df.values, df.values, epochs=100, batch_size=16, shuffle=True,callbacks=[early_stopping])

"""
encoded_data = encoder.predict(dfs[0].values)
Encoded.append(DataFrame(encoded_data,
                         index=dfs[0].index,
                         columns=[f'{name}_{k}th' for k in range(1, n_pcs+1)]))
"""
dfs_reconstructed = autoencoder.predict(dfs[i].values)
mse_train = np.mean((dfs[i].values - dfs_reconstructed) ** 2)


dfs2_reconstructed = autoencoder.predict(dfs2[i].values)
mse_test = np.mean((dfs2[i].values - dfs2_reconstructed) ** 2)

print("name:",name)
print("train mse",mse_train)
print("test mse",mse_test)

Epoch 1/100
98/98 [==============================] - 1s 956us/step - loss: 0.2738
Epoch 2/100
98/98 [==============================] - 0s 968us/step - loss: 0.2479
Epoch 3/100
98/98 [==============================] - 0s 997us/step - loss: 0.2386
Epoch 4/100
98/98 [==============================] - 0s 998us/step - loss: 0.2306
Epoch 5/100
98/98 [==============================] - 0s 910us/step - loss: 0.2235
Epoch 6/100
98/98 [==============================] - 0s 902us/step - loss: 0.2171
Epoch 7/100
98/98 [==============================] - 0s 959us/step - loss: 0.2108
Epoch 8/100
98/98 [==============================] - 0s 998us/step - loss: 0.2045
Epoch 9/100
98/98 [==============================] - 0s 992us/step - loss: 0.1985
Epoch 10/100
98/98 [==============================] - 0s 1ms/step - loss: 0.1928
Epoch 11/100
98/98 [==============================] - 0s 1ms/step - loss: 0.1872
Epoch 12/100
98/98 [==============================] - 0s 975us/step - loss: 0.1816
Epoch 13/100
98/9

In [112]:
i = 1
df = dfs[i]
name = gran_names[i]
n_pcs = n_pcss[i]
input_dim = df.shape[1]

input_layer = keras.layers.Input(shape=(input_dim,))
encoded_1 = keras.layers.Dense(128, activation='tanh')(input_layer)
encoded_2 = keras.layers.Dense(64, activation='tanh')(encoded_1)
encoded_final = keras.layers.Dense(n_pcs, activation='tanh')(encoded_2)

# Symmetric decoder architecture
decoded_1 = keras.layers.Dense(64, activation='tanh')(encoded_final)
decoded_2 = keras.layers.Dense(128, activation='tanh')(decoded_1)
decoded_final = keras.layers.Dense(input_dim, activation='linear')(decoded_2)

autoencoder = keras.Model(input_layer, decoded_final)
encoder = keras.Model(input_layer, encoded_final)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(df.values, df.values, epochs=100, batch_size=16, shuffle=True)


encoder = keras.Model(input_layer, encoded_final)

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)


autoencoder.fit(df.values, df.values, epochs=100, batch_size=16, shuffle=True,callbacks=[early_stopping])


dfs_reconstructed = autoencoder.predict(dfs[i].values)
mse_train = np.mean((dfs[i].values - dfs_reconstructed) ** 2)


dfs2_reconstructed = autoencoder.predict(dfs2[i].values)
mse_test = np.mean((dfs2[i].values - dfs2_reconstructed) ** 2)

print("name:",name)
print("train mse",mse_train)
print("test mse",mse_test)

Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1951
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 0.1099
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0579
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0367
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0309
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0283
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0266
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0261
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0244
Epoch 10/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0238
Epoch 11/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0230
Epoch 12/100
5/5 [==============================] - 0s 1ms/step - loss: 0.0227
Epoch 13/100
5/5 [==============================] - 0s 2ms/st